In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.metrics import *

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[2], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation= 'selu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [8]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [10]:
run_experiment(X,y)

Epoch 1/30
58/58 [==============================] - 10s 174ms/step - loss: 0.6591 - accuracy: 0.5641 - precision: 0.5408 - recall: 0.4650 - auc: 0.6293
Epoch 2/30
58/58 [==============================] - 10s 176ms/step - loss: 0.5298 - accuracy: 0.7295 - precision: 0.7110 - recall: 0.7126 - auc: 0.8131
Epoch 3/30
58/58 [==============================] - 10s 176ms/step - loss: 0.5003 - accuracy: 0.7558 - precision: 0.7253 - recall: 0.7710 - auc: 0.8359
Epoch 4/30
58/58 [==============================] - 10s 175ms/step - loss: 0.4334 - accuracy: 0.8171 - precision: 0.7946 - recall: 0.8224 - auc: 0.8815
Epoch 5/30
58/58 [==============================] - 10s 176ms/step - loss: 0.4116 - accuracy: 0.8302 - precision: 0.8138 - recall: 0.8271 - auc: 0.8944
Epoch 6/30
58/58 [==============================] - 10s 176ms/step - loss: 0.3878 - accuracy: 0.8456 - precision: 0.8509 - recall: 0.8131 - auc: 0.9080
Epoch 7/30
58/58 [==============================] - 10s 176ms/step - loss: 0.3694 - accu

58/58 [==============================] - 11s 185ms/step - loss: 0.3393 - accuracy: 0.8686 - precision_1: 0.8263 - recall_1: 0.9112 - auc_1: 0.9290
Epoch 24/30
58/58 [==============================] - 11s 183ms/step - loss: 0.2653 - accuracy: 0.8992 - precision_1: 0.8668 - recall_1: 0.9276 - auc_1: 0.9525
Epoch 25/30
58/58 [==============================] - 11s 183ms/step - loss: 0.3273 - accuracy: 0.8587 - precision_1: 0.8360 - recall_1: 0.8692 - auc_1: 0.9304
Epoch 26/30
58/58 [==============================] - 11s 182ms/step - loss: 0.3566 - accuracy: 0.8434 - precision_1: 0.8038 - recall_1: 0.8808 - auc_1: 0.9230
Epoch 27/30
58/58 [==============================] - 11s 183ms/step - loss: 0.2725 - accuracy: 0.9069 - precision_1: 0.8871 - recall_1: 0.9182 - auc_1: 0.9535
Epoch 28/30
58/58 [==============================] - 11s 184ms/step - loss: 0.2439 - accuracy: 0.8970 - precision_1: 0.8813 - recall_1: 0.9019 - auc_1: 0.9615
Epoch 29/30
58/58 [==============================] - 10s 1

58/58 [==============================] - 11s 183ms/step - loss: 0.2459 - accuracy: 0.9091 - precision_3: 0.8947 - recall_3: 0.9136 - auc_3: 0.9587
Epoch 15/30
58/58 [==============================] - 11s 182ms/step - loss: 0.4907 - accuracy: 0.7590 - precision_3: 0.7737 - recall_3: 0.6869 - auc_3: 0.8493
Epoch 16/30
58/58 [==============================] - 11s 183ms/step - loss: 0.3107 - accuracy: 0.8675 - precision_3: 0.8612 - recall_3: 0.8551 - auc_3: 0.9392
Epoch 17/30
58/58 [==============================] - 11s 184ms/step - loss: 0.3323 - accuracy: 0.8609 - precision_3: 0.8209 - recall_3: 0.8995 - auc_3: 0.9330
Epoch 18/30
58/58 [==============================] - 11s 184ms/step - loss: 0.4232 - accuracy: 0.8127 - precision_3: 0.7775 - recall_3: 0.8411 - auc_3: 0.8856
Epoch 19/30
58/58 [==============================] - 11s 182ms/step - loss: 0.3102 - accuracy: 0.8751 - precision_3: 0.8413 - recall_3: 0.9042 - auc_3: 0.9400
Epoch 20/30
58/58 [==============================] - 11s 1

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation= 'selu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [12]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [13]:
run_experiment_2(X,y)

Epoch 1/30
58/58 [==============================] - 18s 316ms/step - loss: 0.6374 - accuracy: 0.6156 - precision_5: 0.5928 - recall_5: 0.5748 - auc_5: 0.6689
Epoch 2/30
58/58 [==============================] - 19s 319ms/step - loss: 0.5207 - accuracy: 0.7470 - precision_5: 0.7127 - recall_5: 0.7710 - auc_5: 0.8183
Epoch 3/30
58/58 [==============================] - 19s 321ms/step - loss: 0.4414 - accuracy: 0.8083 - precision_5: 0.7921 - recall_5: 0.8014 - auc_5: 0.8768
Epoch 4/30
58/58 [==============================] - 18s 318ms/step - loss: 0.4305 - accuracy: 0.8171 - precision_5: 0.7973 - recall_5: 0.8178 - auc_5: 0.8833
Epoch 5/30
58/58 [==============================] - 19s 319ms/step - loss: 0.3889 - accuracy: 0.8467 - precision_5: 0.8288 - recall_5: 0.8481 - auc_5: 0.9046
Epoch 6/30
58/58 [==============================] - 19s 320ms/step - loss: 0.3584 - accuracy: 0.8554 - precision_5: 0.8348 - recall_5: 0.8621 - auc_5: 0.9196
Epoch 7/30
58/58 [==============================] - 

58/58 [==============================] - 19s 323ms/step - loss: 0.1333 - accuracy: 0.9595 - precision_6: 0.9600 - recall_6: 0.9533 - auc_6: 0.9864
Epoch 23/30
58/58 [==============================] - 19s 319ms/step - loss: 0.1532 - accuracy: 0.9409 - precision_6: 0.9193 - recall_6: 0.9579 - auc_6: 0.9852
Epoch 24/30
58/58 [==============================] - 19s 324ms/step - loss: 0.0956 - accuracy: 0.9682 - precision_6: 0.9629 - recall_6: 0.9696 - auc_6: 0.9930
Epoch 25/30
58/58 [==============================] - 19s 322ms/step - loss: 0.1224 - accuracy: 0.9595 - precision_6: 0.9374 - recall_6: 0.9790 - auc_6: 0.9882
Epoch 26/30
58/58 [==============================] - 19s 320ms/step - loss: 0.1314 - accuracy: 0.9551 - precision_6: 0.9510 - recall_6: 0.9533 - auc_6: 0.9877
Epoch 27/30
58/58 [==============================] - 18s 316ms/step - loss: 0.3154 - accuracy: 0.8740 - precision_6: 0.8111 - recall_6: 0.9533 - auc_6: 0.9424
Epoch 28/30
58/58 [==============================] - 18s 3

58/58 [==============================] - 19s 319ms/step - loss: 0.3214 - accuracy: 0.8631 - precision_8: 0.8976 - recall_8: 0.7991 - auc_8: 0.9375
Epoch 14/30
58/58 [==============================] - 19s 320ms/step - loss: 0.2732 - accuracy: 0.9014 - precision_8: 0.8841 - recall_8: 0.9089 - auc_8: 0.9533
Epoch 15/30
58/58 [==============================] - 19s 323ms/step - loss: 0.2439 - accuracy: 0.9014 - precision_8: 0.8912 - recall_8: 0.8995 - auc_8: 0.9621
Epoch 16/30
58/58 [==============================] - 18s 318ms/step - loss: 0.2844 - accuracy: 0.8916 - precision_8: 0.8538 - recall_8: 0.9276 - auc_8: 0.9479
Epoch 17/30
58/58 [==============================] - 19s 320ms/step - loss: 0.2413 - accuracy: 0.9146 - precision_8: 0.8941 - recall_8: 0.9276 - auc_8: 0.9602
Epoch 18/30
58/58 [==============================] - 19s 322ms/step - loss: 0.2945 - accuracy: 0.8773 - precision_8: 0.8465 - recall_8: 0.9019 - auc_8: 0.9438
Epoch 19/30
58/58 [==============================] - 18s 3